>[Introduction](#scrollTo=6VPS3eZBYHJa)

>[Objective](#scrollTo=WwcK8_GvZmV5)

>[Selected Pre-train Model and Custom Dataset](#scrollTo=DcdQhKXqaSI7)

>[Fine-tuning process](#scrollTo=rWh8ApgsbrQa)

>>[Check GPU](#scrollTo=LPN6mmIjONSG)

>>[Install important packages](#scrollTo=ajV1gBsGPquO)

>>[Dataset for Instruction-Finetuning](#scrollTo=Hb3M65DhRaqo)

>>[Load Based Model from Huggingface](#scrollTo=Eq6x46-afaXy)

>>[Import requirement libary](#scrollTo=uomAEffbfkj5)

>>[Login to Huggingface](#scrollTo=cGttD3MNfyHP)

>>[Load the Model](#scrollTo=R30sJuT7gJVi)

>>[Finetune](#scrollTo=sLbObW6FiVDt)

>>[Save the adapter we got from trainer](#scrollTo=k7Qa02OHX7XB)

>>[Code for inferencing model (for later use)](#scrollTo=RsujPxvwPPxc)

>[Finetuning Results and Analysis](#scrollTo=Ix4adjIScMlX)

>[Discussion](#scrollTo=M-lEpsdgcazi)

>[Conclusion](#scrollTo=rdkwSSnfcdxq)

>[Resource](#scrollTo=J4nRiHHFT2kj)



# **Introduction**

I used Supervised fine-tuning (SFT) or Instruction tuning in other names

# **Objective**

To find out how finetuning technique can improve model response compared to baseline model, especially LLM that furthur pretrained from low-resource language like Thai language


# **Selected Pre-train Model and Custom Dataset**

- Thai Pretrain model: [Typhoon 7B from SCB10x](https://huggingface.co/scb10x/typhoon-7b)
- Thai Dataset: [Wikithai V3 from Pythainlp](https://huggingface.co/datasets/pythainlp/thai-wiki-dataset-v3/viewer/default/train?row=48)

# **Fine-tuning process**

## **Check GPU**

Since you have to have enough resource for finetuning, things to consider about finetuning 7B parameter LLMs are:
- GPU RAM (VRAM) at least 14 GB requirement, I used V100 GPU from Colab pro, so I got 16 GB VRAM which enough for this case
- System RAM: for finetuning via Google Colab, need to use High RAM choice


**Basic knowledge for this sections:**
1. For modern LLms, the highest precision weight representation is float16 or bfloat16 (16 bits, 2 bytes), if you want to run model in its full original precision, to get the highest quality output and the full capabilities of the model, you need 2 bytes for each of the weight parameter, for this case 7B

          7B parameters = 14x10^9 byte = 14 GB requirement

In [ ]:
! nvidia-smi

Fri Mar  1 02:10:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
! nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-43aacedb-05e5-4632-09f7-8454b76105ca)


## **Install important packages**

- **bitandbytes** package for leverage Quatization technique, QLORA in this case, to load model in 4 bits

**For further reading**
- [QLORA paper](https://arxiv.org/abs/2305.14314)
- [QLORA github](https://github.com/artidoro/qlora)



In [ ]:
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 11.3 MB/s eta 0:00:00


- **transformers** package for loading LLMs, tokenizer of LLMs, and adgust some config, I will use this package for importing following libary:

    - **AutoModelForCausalLM:** this class is used with  ```from_pretrained()``` method to load the model weights
    - **AutoTokenizer:** this class is used with is a method ```from_pretrained()``` method for automatic loading a pre-trained tokenizer based on the specified model architecture and configuration in the Hugging Face Transformers library
    - **BitsAndBytesConfig:** this class allows us to set ```load_in_4bit``` parameter, this flag is used to enable 4-bit quantization by replacing the Linear layers with FP4/NF4 layers
    - **HfArgumentParser:** a utility module that provides command-line argument parsing capabilities for fine-tuning models in the Hugging Face Transformers library, it helps us define arguments that can be passed to your script and parses them according to the specifications we define
    - **TrainingArguments:** this class is used to define and encapsulate all the arguments required for training a model, it's part of the Hugging Face Transformers library and allows you to specify training-related parameters such as the number of epochs, learning rate, batch size, logging intervals, and so on
    

In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


- **PEFT** (Parameter-Efficient Fine-Tuning) is a library for efficiently adapting large pretrained models to various downstream applications without fine-tuning all of a model’s parameters, PEFT is integrated with the Transformers, Diffusers, and Accelerate libraries to provide a faster and easier way to load, train, and use large models for inference.
   - **LoraConfig:** import and set up this to create a LoRA model from a pretrained transformer model
   - **PeftModel:**
   - **prepare_model_for_kbit_training:**
   - **get_peft_model:**
   - **AutoPeftModelForCausalLM:**


In [ ]:
!pip install -q -U git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


- **trl** this package used for import `SFTTrainer` libary
  

In [ ]:
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00


In [4]:
!pip install -q  wandb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
! pip install huggingface-hub==0.19.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
! pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl 

## **Dataset for Instruction-Finetuning**

since I have already make dataset in the format of Mistral format stored in my Google drive, so I just directly load them

In [5]:
from datasets import load_dataset

In [19]:
train_dataset = load_dataset('json', data_files='/content/drive/MyDrive/ThaiDataset/wikiThai_ft_Typhoon.jsonl', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Load Based Model from Huggingface**

In [ ]:
# base model from huggingFace or path to model
base_model = "scb10x/typhoon-7b"
new_model = "Typhoon_wikiThai"


## **Import requirement libary**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
import os,torch
from trl import SFTTrainer
import pandas as pd
from datasets import Dataset

## **Login to Huggingface**


In [ ]:
from google.colab import userdata
secret_hf = userdata.get('hf_for_write_model')
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## **Load the Model**

- Pass a ```BitsAndBytesConfig``` argument that contains the main hyperparameters for 4 bit quantization (store the data in 4-bit )
- The ```bnb_4bit_compute_dtype``` is the hyperparameter that we have to remember for the merging, for this, dequantize the storage data type is occured to the computation data type to perform the forward and backward pass, but we only compute weight gradients for the LoRA parameters which use 16-bit BrainFloat, according to [**QLORA**](https://arxiv.org/abs/2305.14314)paper
(compute the weight gradient for data dequantized, float16 precision)


In [ ]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

## **Finetune**

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/FinetuneTyphoon",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",

)


Alternatively, if you use load_best_model=True in the config as well, and then do trainer.state.best_model_checkpoint after training, you can get the best checkpoint number, and again from that you can infer that the other output directory contains the latest model.

This is not exact, but if you use save_strategy=steps and save_steps=NUMBER, it seems that the total number of steps done during training is approximately the number of steps defined in save_steps multiplied with the batch size defined in per_device_train_batch_size.

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="Text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
pwd

'/content'

In [ ]:
cd /content/drive/MyDrive/FinetuneTyphoon

/content/drive/MyDrive/FinetuneTyphoon


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.685700
2,1.790800
3,1.660600
4,1.435400
5,1.582000
6,1.857400
7,1.602400
8,1.745100
9,1.488700
10,1.717600


Checkpoint destination directory /content/drive/MyDrive/FinetuneTyphoon/checkpoint-50 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory /content/drive/MyDrive/FinetuneTyphoon/checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of 

TrainOutput(global_step=750, training_loss=1.5120778374473254, metrics={'train_runtime': 8112.8496, 'train_samples_per_second': 0.37, 'train_steps_per_second': 0.092, 'total_flos': 9.296844657917952e+16, 'train_loss': 1.5120778374473254, 'epoch': 1.0})

## **Save the adapter we got from trainer**

In [ ]:
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(35219, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): 

In [ ]:
trainer.model.push_to_hub(new_model)

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/radchaneepornc/Typhoon_wikiThai/commit/79f4bb679b5d2d376de351896ae1cb9695e9a848', commit_message='Upload model', commit_description='', oid='79f4bb679b5d2d376de351896ae1cb9695e9a848', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(35219, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): 

In [ ]:
logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

In [ ]:
def build_prompt(question):
  prompt=f"<s>[INST]{question}[/INST]"
  return prompt

In [ ]:
question = "วิศวกรรมเคมี คืออะไร"
prompt = build_prompt(question)
result = pipe(prompt)

print(result[0]['generated_text'])

<s>[INST]วิศวกรรมเคมี คืออะไร[/INST] \n วิศวกรรมเคมี (Chemical Engineering) คือ วิชาที่ศึกษาเกี่ยวกับการออกแบบและควบคุมกระบวนการทางเคมีและกระบวนการทางเคมี-วิศวกรรมเคมี (Chemical Engineering) คือ วิชาที่ศึกษาเกี่ยวกับการออกแบบและควบคุมกระบวนการทางเคมีและกระบวนการทางเคมี-วิศวกรรมเคมี (Chemical Engineering) คือ วิชาที่ศึกษาเกี่ยวกับการออกแบบและควบคุมกระบวนการทางเคมีและกระบวนการทางเคมี-วิศวกรรมเคมี (Chemical Engineering) คือ วิชาที่ศึกษาเกี่ยวกับการออกแบบและควบคุมกระบวนการทางเคมีและกระบวนการทางเคมี-วิศวกรรมเคมี (Chemical Engineering) คือ วิชาที่ศึกษาเกี่ยวกับการออกแบบและควบคุมกระบวนการทางเคมีและกระบวนการทางเคมี-วิศวกรรมเคมี (Chemical Engineering) คือ วิชาที่ศึกษาเกี่ยวกับการออกแบบและควบคุมกระบวนการทางเคมีและกระบวนการทางเคมี-วิศวกรรมเคมี (Chemical Engineering) คือ วิชาที่ศึกษาเกี่ยวกับ


## **Code for inferencing model (for later use)**

In [ ]:
base_model = "scb10x/typhoon-7b"
new_model = "radchaneepornc/Typhoon_wikiThai"

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        return_dict=True,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)
#model + LoRA adaptor
model = PeftModel.from_pretrained(base_model_reload, new_model)
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id


pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/563k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [ ]:
from datasets import load_dataset

In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/ThaiDataset/wikiThai_196533rows.csv")

In [ ]:
data.iloc[20]

title                             การประมวลผลสัญญาณดิจิทัล
text     การประมวลผลสัญญาณดิจิทัล หรือ ที่เรียกกันติดปา...
Name: 20, dtype: object

# **Finetuning Results and Analysis**

In [ ]:
def build_prompt(question):
  prompt=f"<s>[INST]{question} [/INST]"
  return prompt

In [ ]:
%%time
question = 'วันศุกร์คือ อะไร'
prompt = build_prompt(question)
result = pipe(prompt)
llmAnswer = result[0]['generated_text']
llmAnswer

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 1min 38s, sys: 2.79 s, total: 1min 41s
Wall time: 1min 40s


'<s>[INST]วันศุกร์คือ อะไร [/INST] \\n วันศุกร์ (Friday) เป็นวันหนึ่งในสัปดาห์ เป็นวันสุดท้ายของสัปดาห์การทำงาน และเป็นวันแรกของวันหยุดสุดสัปดาห์\n== วันศุกร์ในภาษาต่าง ๆ ==\n== ดูเพิ่ม ==\nรายชื่อวันในสัปดาห์\n== อ้างอิง ==\n== แหล่งข้อมูลอื่น ==\nวันศุกร์\nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ '

In [ ]:
%%time
question = 'วิศวะคอมพิวเตอร์ คืออะไร'
prompt = build_prompt(question)
result = pipe(prompt)
llmAnswer = result[0]['generated_text']
llmAnswer

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 1min 38s, sys: 2.55 s, total: 1min 40s
Wall time: 1min 40s


'<s>[INST]วิศวะคอมพิวเตอร์ คืออะไร [/INST] \\n วิศวกรรมคอมพิวเตอร์ (Computer Engineering) เป็นสาขาวิชาที่ศึกษาเกี่ยวกับการออกแบบและพัฒนาซอฟต์แวร์และฮาร์ดแวร์คอมพิวเตอร์ วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ผสมระหว่างวิศวกรรมไฟฟ้าและวิศวกรรมคอมพิวเตอร์ วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ศึกษาเกี่ยวกับการออกแบบและพัฒนาซอฟต์แวร์และฮาร์ดแวร์คอมพิวเตอร์ วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ผสมระหว่างวิศวกรรมไฟฟ้าและวิศวกรรมคอมพิวเตอร์ วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ศึกษาเกี่ยวกับการออกแบบและพัฒนาซอฟต์แวร์และฮาร์ดแวร์คอมพิวเตอร์ วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ผสมระหว่างวิศวกรรมไฟฟ้าและวิศวกรรมคอมพิวเตอร์ วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ศึกษาเกี่ยวกับการออกแบบและพัฒนาซอฟต์แวร์และฮาร์ดแวร์คอมพิวเตอร์ วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ผสมระหว่างวิศวกรรมไฟฟ้าและวิศวกรรมคอมพิวเตอร์ วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชา'

In [ ]:
%%time
question = 'ดาราศาสตร์คืออะไร'
prompt = build_prompt(question)
result = pipe(prompt)
llmAnswer = result[0]['generated_text']
llmAnswer

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 1min 38s, sys: 2.6 s, total: 1min 40s
Wall time: 1min 40s


'<s>[INST]ดาราศาสตร์คืออะไร [/INST] \\n ดาราศาสตร์ (astronomy) คือการศึกษาเกี่ยวกับวัตถุท้องฟ้า เช่น ดาวฤกษ์ ดาวเคราะห์ ดาวหาง ดาวตก ดาวตกสะเก็ด ดาวตกหาง ดาวเคราะห์น้อย ดาวเคราะห์แคระ ดาวนิวตรอน ดาวนิวตรอนขาว ดาวนิวตรอนสีฟ้า ดาวนิวตรอนสีแดง ดาราจักร กาแลกซี่ วัตถุท้องฟ้าอื่น ๆ\n== ดูเพิ่ม ==\nดาราศาสตร์\n== แหล่งข้อมูลอื่น ==\nดาราศาสตร์ \nวิทยาศาสตร์ \nวิชาดาราศาสตร์ \nวิชาวิทยาศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ \nวิชาดาราศาสตร์ '

In [ ]:
%%time
question = 'วิศวะเคมีทำหน้าที่อะไร'
prompt = build_prompt(question)
result = pipe(prompt)
llmAnswer = result[0]['generated_text']
llmAnswer

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 1min 37s, sys: 2.77 s, total: 1min 40s
Wall time: 1min 39s


'<s>[INST]วิศวะเคมีทำหน้าที่อะไร [/INST] \\n วิศวกรรมเคมี (Chemical Engineering) เป็นสาขาวิชาที่ศึกษาเกี่ยวกับการออกแบบและควบคุมกระบวนการทางเคมีและกระบวนการทางเคมีที่เกี่ยวข้องกับกระบวนการทางเคมี เช่น การผลิตปิโตรเลียม การผลิตปุ๋ย การผลิตสารเคมี การผลิตยา การผลิตอาหาร การผลิตกระดาษ การผลิตพลาสติก การผลิตโลหะ การผลิตสารกึ่งตัวนำ การผลิตสารเคมีอันตราย การผลิตสารเคมีที่ใช้ในอุตสาหกรรม การผลิตสารเคมีที่ใช้ในอุตสาหกรรมอาหาร การผลิตสารเคมีที่ใช้ในอุตสาหกรรมยา การผลิตสารเคมีที่ใช้ในอุตสาหกรรมพลาสติก การผลิตสารเคมีที่ใช้ในอุตสาหกรรมกระดาษ การผลิตสารเคมีที่ใช้ในอุตสาหกรรมโลหะ การผลิตสารเคมีที่ใช้ในอุตสาหกรรมสารกึ่งตัวนำ การผลิตสารเคมีที่ใช้ในอุตสาหกรรมสารเคมีอันตราย การผลิตสารเคมีที่ใช้ในอุตสาหกรรมสารเคมีที่ใช้ในอุตสาหกรรมอาหาร การผลิตสารเคมีที่ใช้ในอุตสาหกรรมยา การผลิตสารเคมีที่ใช้ในอุตสาหกรรมพลาสติก การผลิตสารเคมีที่ใช้ใน'

In [ ]:
%%time
question = 'วิศวะเคมีทำหน้าที่อะไร'
prompt = build_prompt(question)
result = pipe(prompt)
result

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 1min 36s, sys: 2.6 s, total: 1min 39s
Wall time: 1min 38s


[{'generated_text': '<s>[INST]วิศวะเคมีทำหน้าที่อะไร [/INST] \\n วิศวกรรมเคมี (Chemical Engineering) เป็นสาขาวิชาที่ศึกษาเกี่ยวกับการออกแบบและควบคุมกระบวนการทางเคมีและกระบวนการทางเคมีที่เกี่ยวข้องกับกระบวนการทางเคมี เช่น การผลิตปิโตรเลียม การผลิตปุ๋ย การผลิตสารเคมี การผลิตยา การผลิตอาหาร การผลิตกระดาษ การผลิตพลาสติก การผลิตโลหะ การผลิตสารกึ่งตัวนำ การผลิตสารเคมีอันตราย การผลิตสารเคมีที่ใช้ในอุตสาหกรรม การผลิตสารเคมีที่ใช้ในอุตสาหกรรมอาหาร การผลิตสารเคมีที่ใช้ในอุตสาหกรรมยา การผลิตสารเคมีที่ใช้ในอุตสาหกรรมพลาสติก การผลิตสารเคมีที่ใช้ในอุตสาหกรรมกระดาษ การผลิตสารเคมีที่ใช้ในอุตสาหกรรมโลหะ การผลิตสารเคมีที่ใช้ในอุตสาหกรรมสารกึ่งตัวนำ การผลิตสารเคมีที่ใช้ในอุตสาหกรรมสารเคมีอันตราย การผลิตสารเคมีที่ใช้ในอุตสาหกรรมสารเคมีที่ใช้ในอุตสาหกรรมอาหาร การผลิตสารเคมีที่ใช้ในอุตสาหกรรมยา การผลิตสารเคมีที่ใช้ในอุตสาหกรรมพลาสติก การผลิตสารเคมีที่ใช้ใน'}]

In [ ]:
import gradio as gr

In [ ]:
def chatbot(user_input):
    if user_input.lower() == 'exit':
        return "Exiting."
    return f"{pipe(user_input)[0]['generated_text']}"

In [ ]:
iface = gr.Interface(fn=chatbot, inputs="text", outputs= "text")
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c65feb83fd108b381c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### **Comparing answer from finetuned model with off-the-shelf model, Typhoon in this case**

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model = "scb10x/typhoon-7b"
finetune_adaptor = "radchaneepornc/Typhoon_wikiThai"

# Load chatbot models and tokenizers for off-the-selve model
tokenizer_model1 = AutoTokenizer.from_pretrained(base_model)
model1 = AutoModelForCausalLM.from_pretrained(base_model)


base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        return_dict=True,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)
#model + LoRA adaptor
model = PeftModel.from_pretrained(base_model_reload, finetune_adaptor)

#make pipeline, use tokenizer same with model1
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer_model1, max_length=200)

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/563k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [ ]:
input_text=input()
pipe_typhoon = pipeline(task="text-generation", model=model1, tokenizer=tokenizer_model1, max_length=200)
response1 = pipe_typhoon(f"<s>[INST]{input_text}[/INST]")
response2 = pipe(f"<s>[INST]{input_text}[/INST]")

print(f"This is response from Typhoon:\n {response1}")
print(f"This is response from Typhoon finetuned with Thaiwiki dataset:\n {response2}")

ดาราศาสตร์ คืออะไร


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


This is response from Typhoon:
 [{'generated_text': '<s>[INST]ดาราศาสตร์ คืออะไร[/INST]\n[INST]ดาราศาสตร์ คืออะไร[/INST]\nดาราศาสตร์ คือ การศึกษาเกี่ยวกับวัตถุในอวกาศ เช่น ดาวเคราะห์ ดวงดาว ดาวฤกษ์ ดาวหาง ดาวเคราะห์น้อย และอื่นๆ\nดาราศาสตร์ เป็นวิชาที่ศึกษาเกี่ยวกับวัตถุในอวกาศ เช่น ดาวเคราะห์ ดวงดาว ดาวฤกษ์ ดาวหาง ดาวเคราะห์น้อย และอื่นๆ\nดาราศาสตร์ เป็นวิชาที่ศึกษาเกี่ยวกับวัตถุในอวกาศ เช่น ดาวเคราะห์ ดวงดาว ดาวฤกษ์ ดาวหาง ดาวเคราะห์น้อย และอื่นๆ\nดาราศาสตร์ เป็นวิชาที่ศึกษาเกี่ยวกับวัตถุในอวกาศ เช่น ดาวเคราะห์ ดวงดาว ดาวฤกษ์ ดาวหาง ดาวเคราะห์น้อย และอื่นๆ\nดาราศาสตร์ เป็นวิชาที่ศึกษาเกี่ยวกับวัตถุในอวกาศ เช่น ดาวเคราะห์ ดวงดาว ดาวฤกษ์ ดาวหาง ดาวเคราะห์น้อย และอื่นๆ\nดาราศาสตร์ เป็นวิชาที่ศึกษาเกี่ยวกับวัตถุในอวกาศ เช่น'}]
This is response from Typhoon finetuned with Thaiwiki dataset:
 [{'generated_text': '<s>[INST]ดาราศาสตร์ คืออะไร[/INST] \\n ดาราศาสตร์ (astronomy) คือการศึกษาเกี่ยวกับวัตถุท้องฟ้า เช่น ดาวฤกษ์ ดาวเคราะห์ ดาวหาง ดาวตก ดาวตกสะเก็ด ดาวตกหาง ดาวเคราะห์น้อย ดาวเคราะห์แค

In [ ]:
input_text=input()
pipe_typhoon = pipeline(task="text-generation", model=model1, tokenizer=tokenizer_model1, max_length=200)
response1 = pipe_typhoon(f"<s>[INST]{input_text}[/INST]")
response2 = pipe(f"<s>[INST]{input_text}[/INST]")

print(f"This is response from Typhoon:\n {response1}['generated_text']")
print(f"This is response from Typhoon finetuned with Thaiwiki dataset:\n {response2}['generated_text']")

วิศวะคอมพิวเตอร์ คืออะไร


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


This is response from Typhoon:
 [{'generated_text': '<s>[INST]วิศวะคอมพิวเตอร์ คืออะไร[/INST]\nวิศวะคอมพิวเตอร์ คืออะไร\nวิศวกรรมคอมพิวเตอร์ (อังกฤษ: Computer Engineering) เป็นสาขาวิชาที่ศึกษาเกี่ยวกับการประยุกต์ใช้ความรู้ทางด้านวิศวกรรมศาสตร์และคณิตศาสตร์ เพื่อสร้างอุปกรณ์อิเล็กทรอนิกส์และซอฟต์แวร์ที่ทำงานร่วมกัน วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ผสมผสานระหว่างวิศวกรรมไฟฟ้าและวิทยาการคอมพิวเตอร์เข้าด้วยกัน วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่เน้นการประยุกต์ใช้ความรู้ทางด้านวิศวกรรมศาสตร์และคณิตศาสตร์ เพื่อสร้างอุปกรณ์อิเล็กทรอนิกส์และซอฟต์แวร์ที่ทำงานร่วมกัน วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่ผสมผสานระหว่างวิศวกรรมไฟฟ้าและวิทยาการคอมพิวเตอร์เข้าด้วยกัน วิศวกรรมคอมพิวเตอร์เป็นสาขาวิชาที่เน้นการประยุกต์ใช้ความรู้ทางด้านวิศวกรรมศาสตร์และคณิตศาสตร์ เพื่อสร้างอุปกรณ์อิเล็กทรอนิกส์และซอฟต์แวร์ที่ทำงานร่วมกัน วิ'}]['generated_text']
This is response from Typhoon finetuned with Thaiwiki dataset:
 [{'generated_text': '<s>[INST]วิศวะคอมพิวเตอร์ คืออะไร[/INST] \\n วิศวกรรมคอมพิวเตอร์ (computer engi

In [ ]:
%%time
input_text=input()
pipe_typhoon = pipeline(task="text-generation", model=model1, tokenizer=tokenizer_model1, max_length=200)
response1 = pipe_typhoon(f"<s>[INST]{input_text}[/INST]")
response2 = pipe(f"<s>[INST]{input_text}[/INST]")

print(f"This is response from Typhoon:\n {response1}['generated_text']")
print(f"This is response from Typhoon finetuned with Thaiwiki dataset:\n {response2}['generated_text']")

วิศวะเคมี คืออะไร


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


This is response from Typhoon:
 [{'generated_text': '<s>[INST]วิศวะเคมี คืออะไร[/INST]\nวิศวะเคมี คืออะไร\nวิศวกรรมเคมี (อังกฤษ: Chemical Engineering) เป็นสาขาวิชาที่ศึกษาเกี่ยวกับการนำหลักการทางวิทยาศาสตร์และเทคโนโลยีมาประยุกต์ใช้ในการผลิตและแปรรูปสารเคมีและผลิตภัณฑ์ต่างๆ วิศวกรรมเคมีเป็นสาขาวิชาที่ผสมผสานระหว่างวิทยาศาสตร์และวิศวกรรมศาสตร์เข้าด้วยกัน โดยเน้นการประยุกต์ใช้ความรู้ทางเคมีและวิศวกรรมศาสตร์เพื่อการออกแบบและพัฒนากระบวนการผลิตและกระบวนการแปรรูปสารเคมีและผลิตภัณฑ์ต่างๆ วิศวกรรมเคมีเป็นสาขาวิชาที่เน้นการประยุกต์ใช้ความรู้ทางเคมีและวิศวกรรมศาสตร์เพื่อการออกแบบและพัฒนากระบวนการผลิตและกระบวนการแปรรูปสารเคมีและผลิตภัณฑ์ต่างๆ วิศวกรรมเคมีเป็นสาขาวิชาที่เน้นการประยุกต์ใช้ความรู้ทางเคมีและวิศวกรรมศาสตร์เพื่อการออกแบบและพัฒนากระบวนการผลิตและกระบวนการแปรรูปสารเคมีและผลิตภัณฑ์ต่างๆ วิศวกรรม'}]['generated_text']
This is response from Typhoon finetuned with Thaiwiki dataset:
 [{'generated_text': '<s>[INST]วิศวะเคมี คืออะไร[/INST] \\n วิศวกรรมเคมี (Chemical Engineering) คือ วิศวกรรมสาขาหน

In [ ]:
%%time
input_text=input()
pipe_typhoon = pipeline(task="text-generation", model=model1, tokenizer=tokenizer_model1, max_length=200)
response1 = pipe_typhoon(f"<s>[INST]{input_text}[/INST]")
response2 = pipe(f"<s>[INST]{input_text}[/INST]")

print(f"This is response from Typhoon:\n {response1}['generated_text']")
print(f"This is response from Typhoon finetuned with Thaiwiki dataset:\n {response2}['generated_text']")

การประมวลผลสัญญาณดิจิทัล คืออะไร


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


This is response from Typhoon:
 [{'generated_text': '<s>[INST]การประมวลผลสัญญาณดิจิทัล คืออะไร[/INST]\nการประมวลผลสัญญาณดิจิทัล คืออะไร\nการประมวลผลสัญญาณดิจิทัล คืออะไร\nการประมวลผลสัญญาณดิจิทัล คือ การแปลงสัญญาณดิจิทัลให้เป็นสัญญาณอนาล็อก และสัญญาณอนาล็อกให้เป็นสัญญาณดิจิทัล\nการประมวลผลสัญญาณดิจิทัล คือ การแปลงสัญญาณดิจิทัลให้เป็นสัญญาณอนาล็อก และสัญญาณอนาล็อกให้เป็นสัญญาณดิจิทัล\nร่วมเป็นแฟนเพจเรา บน Facebook..ได้ที่นี่เลย!! ▼ กำลังโหลดข้อมูล... ▼\nแสดงความคิดเห็น\nกระทู้ที่คุณอาจสนใจ\nอยากทราบว่ามีใครเคยใช้บริการของบริษัทนี้ไหมคะ\nอยากทราบว่ามีใครเคยใช้บริการของบริษัทนี้ไหมคะ พอดีเราได้ไปสมัครงานกับบริษัทนี้มาค่ะ แล้วเขาให้เราไปทำใบรับรองแพทย์กับโรงพยาบาลที่เขาแนะนำมาให้ค่ะ พอเราไปทำใบรับรองแพทย์เสร็จแล้ว'}]['generated_text']
This is response from Typhoon finetuned with Thaiwiki dataset:
 [{'generated_text': '<s>[INST]การประมวลผลสัญญาณดิจิทัล คืออะไร[/INST] \\n การประมวลผลสัญญาณดิจิทัล (digital signal processing) หรือเรียกย่อ ๆ ว่า การประมวลผลสัญญาณ (signal processing) คือ การประม

In [ ]:
%%time
input_text=input()
pipe_typhoon = pipeline(task="text-generation", model=model1, tokenizer=tokenizer_model1, max_length=200)
response1 = pipe_typhoon(f"<s>[INST]{input_text}[/INST]")
response2 = pipe(f"<s>[INST]{input_text}[/INST]")

print(f"This is response from Typhoon:\n {response1}['generated_text']")
print(f"This is response from Typhoon finetuned with Thaiwiki dataset:\n {response2}['generated_text']")

วันศุกร์คืออะไร


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


This is response from Typhoon:
 [{'generated_text': '<s>[INST]วันศุกร์คืออะไร[/INST]\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุกร์คืออะไร\nวันศุก'}]['generated_text']
This is response from Typhoon finetuned with Thaiwiki dataset:
 [{'generated_text': '<s>[INST]วันศุกร์คืออะไร[/INST] \\n วันศุกร์ (Friday) เป็นวันหนึ่งในสัปดาห์ เป็นวันที่ 5 ของสัปดาห์ ตามหลักโหราศาสตร์ เป็นวันสุดท้ายของสัปดาห์ และเป็นวันสุดท้ายของวันทำงานของชาวตะวันตก\n== วันศุกร์ในภาษาต่าง ๆ ==\n== ดูเพิ่ม ==\nรายชื่อวันในสัปดาห์\n== อ้างอิง ==\n== แหล่งข้อมูลอื่น ==\nวันศุกร์\nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุกร์ \nวันศุก

In [ ]:
%%time
input_text=input()
pipe_typhoon = pipeline(task="text-generation", model=model1, tokenizer=tokenizer_model1, max_length=200)
response1 = pipe_typhoon(f"<s>[INST]{input_text}[/INST]")
response2 = pipe(f"<s>[INST]{input_text}[/INST]")

print(f"This is response from Typhoon:\n {response1}['generated_text']")
print(f"This is response from Typhoon finetuned with Thaiwiki dataset:\n {response2}['generated_text']")

วิทยาการคอมพิวเตอร์ คืออะไร


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


This is response from Typhoon:
 [{'generated_text': '<s>[INST]วิทยาการคอมพิวเตอร์ คืออะไร[/INST]\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\nวิทยาการคอมพิวเตอร์ คืออะไร\n'}]

# **Resource**
**For Finetuning tutorial**

- https://medium.com/@codersama/fine-tuning-mistral-7b-in-google-colab-with-qlora-complete-guide-60e12d437cca
- https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Mistral/Supervised_fine_tuning_(SFT)_of_an_LLM_using_Hugging_Face_tooling.ipynb
- https://kaitchup.substack.com/p/lora-adapters-when-a-naive-merge?utm_source=%2Fsearch%2Fqlora&utm_medium=reader2

**For original dataset**
- https://huggingface.co/datasets/pythainlp/thai-wiki-dataset-v3